In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

In [2]:
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = user_secrets.get_secret("LANGSMITH_API_KEY")
os.environ["LANGSMIpTH_PROJECT"] = "langchain kaggle"
os.environ["MISTRAL_API_KEY"] = user_secrets.get_secret("mistralAgent")

In [13]:
!pip install langchain langchain-community langchain-mistralai langgraph pydantic pypdf2 python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 12.1 MB/s eta 0:00:00


In [27]:
import os
from typing import TypedDict, List
from langchain_mistralai import ChatMistralAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from pydantic import BaseModel, Field


# Initialize Mistral LLM
llm = ChatMistralAI(
    model="mistral-large-latest",  # Or other models like 'open-mistral-nemo', 'mistral-small-latest'
    temperature=0.2
)

# Define Pydantic model for structured grading response (no dict for marks; use specific fields for fixed questions)
class GradingResult(BaseModel):
    """Structured grading output for the two questions."""
    q1_marks: str = Field(description="Marks for Question 1, e.g., '38'")
    q2_marks: str = Field(description="Marks for Question 2, e.g., '35'")
    remarks: str = Field(description="Overall brief remarks")

# Bind the LLM with structured output using Pydantic model (uses JSON mode under the hood)
structured_llm = llm.with_structured_output(GradingResult)

questions_rubrics = """
Question 1: You are building an employee management system for a company that needs to handle different types of employees with varying capabilities.
Requirements:
Create the following classes to demonstrate inheritance, abstract classes, and interfaces:
■ Interface IManager
○ Method: ManageTeam() returns string
■ Base class Employee
○ Properties: Name (string), Salary (decimal)
○ Virtual method: Work() returns "Working on tasks"
■ Class Developer inherits Employee and implements IManager
○ Override Work() to return "Writing code"
○ Implement ManageTeam() to return "Leading development team"
■ Abstract class SalesEmployee inherits Employee
○ Abstract method: MakeSale() returns string
■ Class SalesRep inherits SalesEmployee
○ Override Work() to return "Meeting clients"
○ Implement MakeSale() to return "Closed a deal"

Rubrics for Question 1:

| **Criteria**                          | **Description**                                                                                                                                              | **Marks** |
| ------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------ | --------- |
| **Interface Implementation**          | Correctly defines `IManager` interface with method `ManageTeam()` returning string.                                                                          | 5         |
| **Base Class Employee**               | Includes properties `Name` (string) and `Salary` (decimal). Declares virtual `Work()` method returning `"Working on tasks"`.                                 | 10        |
| **Developer Class**                   | Inherits `Employee`, implements `IManager`. Overrides `Work()` to return `"Writing code"`. Implements `ManageTeam()` to return `"Leading development team"`. | 10        |
| **Abstract Class SalesEmployee**      | Properly inherits `Employee` and declares abstract method `MakeSale()` returning string.                                                                     | 7         |
| **SalesRep Class**                    | Inherits `SalesEmployee`. Overrides `Work()` to return `"Meeting clients"`. Implements `MakeSale()` to return `"Closed a deal"`.                             | 10        |
| **Code Structure & Syntax**           | Proper use of `public`, `override`, `abstract`, and `interface`. Correct C# syntax, indentation, no logical errors.                                          | 5         |
| **Overall OOP Concepts Demonstrated** | Shows understanding of inheritance, abstraction, interface implementation, and polymorphism.                                                                 | 3         |

Question 2: Patient model containing properties: Id (int), Name (string), Age (int), BedNo(string)
1. Name is max 30 characters.
2. Age should be non-negative
3. BedNo is just 4 characters
Task:
Create a model with data annotations: Patient.cs (model code)
Create a list of patients and write a query to get the names of 5 oldest patients.

Rubrics for Question 2:

| **Criteria**                                 | **Description**                                                                                      | **Marks** |
| -------------------------------------------- | ---------------------------------------------------------------------------------------------------- | --------- |
| **Model Definition**                         | Defines class `Patient` with properties: `Id (int)`, `Name (string)`, `Age (int)`, `BedNo (string)`. | 5         |
| **Name Validation**                          | Uses `[MaxLength(30)]` or `[StringLength(30)]` annotation on `Name`.                                 | 5         |
| **Age Validation**                           | Ensures `Age` is non-negative using `[Range(0, int.MaxValue)]`.                                      | 7         |
| **BedNo Validation**                         | Uses `[StringLength(4)]` to enforce exactly 4 characters (or appropriate validation).                | 8         |
| **List Creation**                            | Creates a sample list of patients (with multiple entries).                                           | 5         |
| **LINQ Query – Oldest 5 Patients**           | Correct query to order patients by `Age` descending and select top 5 names.                          | 10        |
| **Code Readability & Structure**             | Clean indentation, meaningful naming, proper use of `List<Patient>`.                                 | 5         |
| **Understanding of Data Annotations & LINQ** | Demonstrates correct integration of model validation + query writing.                                | 5         |
"""
# Add more for additional questions if needed.

# Prompt template for grading (updated to match Pydantic model structure)
grading_prompt = ChatPromptTemplate.from_template(
    """You are an expert grader. Evaluate the student's C# code/answers based on the given questions and rubrics.

    Questions and Rubrics:
    {questions_rubrics}

    Student Answers/Code:
    {student_answers}
   Grading Instructions:
- Award partial marks where the student demonstrates partial understanding or correct reasoning, even if the final code/answer is incorrect.
- If something not asked in the question is included, do not penalize if it is correct or relevant.
- Be constructive and avoid being overly strict on syntax mistakes if the core logic is correct.
    """
)

# Define the grading chain (using structured LLM)
grading_chain = grading_prompt | structured_llm

# For LangGraph: Define a simple stateful graph for grading workflow
class GradingState(TypedDict):
    student_answers: str
    graded_result: GradingResult  # Use Pydantic model here
    continue_grading: bool  # To control loop

# Node: Grade the input
def grade_node(state: GradingState) -> GradingState:
    if not state.get("student_answers"):
        return {"continue_grading": False}
    graded = grading_chain.invoke({
        "questions_rubrics": questions_rubrics,
        "student_answers": state["student_answers"]
    })
    return {"graded_result": graded, "continue_grading": True}

# Node: Output and check for continue (in notebook, we'll handle loop externally)
def output_node(state: GradingState) -> GradingState:
    print("Grading Result:")
    print(f"Q1 Marks: {state['graded_result'].q1_marks}")
    print(f"Q2 Marks: {state['graded_result'].q2_marks}")
    print(f"Remarks: {state['graded_result'].remarks}")
    return state

# Build the graph
workflow = StateGraph(state_schema=GradingState)
workflow.add_node("grade", grade_node)
workflow.add_node("output", output_node)
workflow.add_edge("grade", "output")
workflow.add_edge("output", END)
workflow.set_entry_point("grade")

# Compile the graph into a runnable app
grading_app = workflow.compile()

In [26]:
import os
from typing import TypedDict, List
from langchain_mistralai import ChatMistralAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from pydantic import BaseModel, Field
from PyPDF2 import PdfReader  # For PDFs
from docx import Document  # For .docx files


# Initialize Mistral LLM
llm = ChatMistralAI(
    model="mistral-large-latest",  # Or other models like 'open-mistral-nemo', 'mistral-small-latest'
    temperature=0.2
)

class GradingResult(BaseModel):
    """Structured grading output for the two questions, including extracted student ID."""
    student_id: str = Field(description="Extracted student ID from the file content, e.g., '22103062'")
    q1_marks: str = Field(description="Marks for Question 1, e.g., '38'")
    q2_marks: str = Field(description="Marks for Question 2, e.g., '35'")
    remarks: str = Field(description="Overall mistakes in one line")
    
# Bind the LLM with structured output using Pydantic model (uses JSON mode under the hood)
structured_llm = llm.with_structured_output(GradingResult)

# Pre-saved rubrics for multiple questions, formatted as multiline string with labeled markdown tables and question descriptions
questions_rubrics = """
Question 1: You are creating an animal management system for a zoo that needs to handle different types of
animals with their specific behaviors.
Requirements:
Create the following classes to demonstrate inheritance, abstract classes, and interfaces:
1. Interface IPet
○ Method: Play() returns string
2. Base class Animal
○ Properties: Name (string), Age (int)
○ Virtual method: MakeSound() returns &quot;Some sound&quot;
3. Class Dog inherits Animal and implements IPet
○ Override MakeSound() to return &quot;Woof&quot;
○ Implement Play() to return &quot;Playing fetch&quot;
4. Abstract class Bird inherits Animal
○ Abstract method: Fly() returns string
5. Class Sparrow inherits Bird
○ Override MakeSound() to return &quot;Chirp&quot;
○ Implement Fly() to return &quot;Flies quickly&quot;

Rubrics for Question 1:


| **Criteria**                  | **Description**                                                                                                                       | **Marks** |
| ----------------------------- | ------------------------------------------------------------------------------------------------------------------------------------- | --------- |
| **Interface IPet**            | Defines `IPet` interface with method `Play()` returning string.                                                                       | 5         |
| **Base Class Animal**         | Defines properties `Name (string)` and `Age (int)`. Includes virtual method `MakeSound()` returning `"Some sound"`.                   | 10        |
| **Dog Class**                 | Inherits `Animal` and implements `IPet`. Overrides `MakeSound()` to return `"Woof"`. Implements `Play()` to return `"Playing fetch"`. | 10        |
| **Abstract Class Bird**       | Inherits `Animal`. Declares abstract method `Fly()` returning string.                                                                 | 7         |
| **Sparrow Class**             | Inherits `Bird`. Overrides `MakeSound()` to return `"Chirp"`. Implements `Fly()` to return `"Flies quickly"`.                         | 10        |
| **Code Structure & Syntax**   | Correct use of `public`, `override`, `abstract`, and `interface`. Proper indentation, clean syntax.                                   | 5         |
| **OOP Concepts Demonstrated** | Shows correct use of polymorphism, abstraction, and interfaces.                                                                       | 3         |


Question 2: Student model containing properties: Id (int), Name (string), Marks(double), Section(string)
1. Name is max 30 characters.
2. Marks is between 0 and 100.
3. Section is just 1 character

Task:
Create a model with data annotations: Student.cs (model code)
Create a list of students and write a query to get 5 students with top marks.

Rubrics for Question 2:

| **Criteria**                                 | **Description**                                                                                             | **Marks** |
| -------------------------------------------- | ----------------------------------------------------------------------------------------------------------- | --------- |
| **Model Definition**                         | Defines `Student` class with properties: `Id (int)`, `Name (string)`, `Marks (double)`, `Section (string)`. | 5         |
| **Name Validation**                          | Applies `[MaxLength(30)]` or `[StringLength(30)]` on `Name`.                                                | 5         |
| **Marks Validation**                         | Applies `[Range(0, 100)]` to ensure valid marks.                                                            | 8         |
| **Section Validation**                       | Applies `[StringLength(1)]` to restrict section to a single character.                                      | 7         |
| **List Creation**                            | Creates a list of students (`List<Student>`) with sample data.                                              | 5         |
| **LINQ Query – Top 5 Students**              | Correctly writes a LINQ query to order by `Marks` descending and select top 5 students.                     | 10        |
| **Code Structure & Readability**             | Proper indentation, naming conventions, well-structured LINQ.                                               | 5         |
| **Understanding of Data Annotations + LINQ** | Demonstrates proper use of validation attributes and querying.                                              | 5         |

"""
# Add more for additional questions if needed.

# Prompt template for grading (updated to match Pydantic model structure)
grading_prompt = ChatPromptTemplate.from_template(
    """You are an expert grader. Evaluate the student's C# code/answers based on the given questions and rubrics.

    Questions and Rubrics:
    {questions_rubrics}

    Student Answers/Code:
    {student_answers}
   Grading Instructions:
- Award partial marks where the student demonstrates partial understanding or correct reasoning, even if the final code/answer is incorrect.
- If something not asked in the question is included, do not penalize if it is correct or relevant.
- Be constructive and avoid being overly strict on syntax mistakes if the core logic is correct.
    """
)

# Define the grading chain (using structured LLM)
grading_chain = grading_prompt | structured_llm

# Function to extract text from file (handles PDF and DOCX)
def extract_text_from_file(file_path):
    if file_path.lower().endswith('.pdf'):
        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() or ""
        return text
    elif file_path.lower().endswith('.docx') or file_path.lower().endswith('.doc'):
        doc = Document(file_path)
        text = "\n".join([para.text for para in doc.paragraphs])
        return text
    else:
        raise ValueError(f"Unsupported file type: {file_path}")

# Input directory
input_dir = "/kaggle/input/labanswers/B/B"

# List all PDF/DOC files
student_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.pdf', '.docx', '.doc'))]

# Process each file and collect grades
grading_results = {}

for file_name in student_files:
    file_path = os.path.join(input_dir, file_name)
    try:
        student_answers = extract_text_from_file(file_path)
        graded = grading_chain.invoke({
            "questions_rubrics": questions_rubrics,
            "student_answers": student_answers
        })
        grading_results[file_name] = {
            "student_id": graded.student_id,
            "q1_marks": graded.q1_marks,
            "q2_marks": graded.q2_marks,
            "remarks": graded.remarks
        }
    except Exception as e:
        grading_results[file_name] = {"error": str(e)}

# Output the results (print and save to CSV, now with student_id column)
import pandas as pd

df = pd.DataFrame.from_dict(grading_results, orient='index')
df.to_csv('/kaggle/working/grading_results.csv', index=True)
print(df)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3615129789.py, line 141)

In [28]:
# Interactive loop cell: Run this to start a grading loop
# Paste student answers when prompted; enter 'quit' to stop

while True:
    student_answers = input("Paste the student's answers here (or 'quit' to stop): ")
    if student_answers.lower() == 'quit':
        break
    
    # Run the graph with the pasted input
    state = {"student_answers": student_answers, "continue_grading": True}
    result = grading_app.invoke(state)
    
    # The output_node already prints, but you can access result['graded_result'] if needed

Paste the student's answers here (or 'quit' to stop):  Task 1 Code: public interface IManager { Public string ManageTeam(); }  public class Employee { public string Name{get;set;} public decimal Salary{get;set;}  public virtual string Work() { return &quot;Working on task&quot;; } }  public class Developer :Employee, IManager { public override string Work() {  return &quot;Writing code&quot;; } public string ManageTeam() { return &quot;Leading development team&quot;; } }  public abstract class SalesEmployee: Employee { public abstract string MakeSales(); } public class SalesRep: SalesEmployee { public override string Work() { return &quot;Meeting clients&quot;; } public override string MakeSales() { return &quot;Closed a deal&quot;; } }  Task 2 Code: public class Patient { public int Id{get;set;} [Requared] [StringLength(30)] public int Id{get;set;} [Requared] [Range(0)] public int Id{get;set;} [Requared] [StringLength(4,MinimumLength=4)] public BedNo {get;set;}  }  public class HelloW

Grading Result:
Q1 Marks: 35
Q2 Marks: 18
Remarks: Question 1: The student demonstrated a good understanding of inheritance, abstraction, and interface implementation but had minor syntax errors (e.g., 'Public' instead of 'public', 'MakeSales' instead of 'MakeSale'). Question 2: The model definition and data annotations were incorrect due to repeated properties, invalid attribute names (e.g., 'Requared' instead of 'Required'), and logical errors in the LINQ query. The LINQ query also had syntax mistakes (e.g., 'OrderByDecending' instead of 'OrderByDescending', 'Takes' instead of 'Take').


KeyboardInterrupt: Interrupted by user

In [19]:
for file_name in student_files:
    file_path = os.path.join(input_dir, file_name)
    try:
        student_answers = extract_text_from_file(file_path)
        graded = grading_chain.invoke({
            "questions_rubrics": questions_rubrics,
            "student_answers": student_answers
        })
        grading_results[file_name] = {
            "student_id": graded.student_id,
            "q1_marks": graded.q1_marks,
            "q2_marks": graded.q2_marks,
            "remarks": graded.remarks
        }
    except Exception as e:
        grading_results[file_name] = {"error": str(e)}

# Output the results (print and save to CSV, now with student_id column)
import pandas as pd

df = pd.DataFrame.from_dict(grading_results, orient='index')
df.to_csv('/kaggle/working/grading_results.csv', index=True)
print(df)


                                                                        student_id  \
Joynabkhatun_22103435.docx                                                22103435   
setuc(1).docx                                                             22103324   
SahidulIslam.docx                                                         22103374   
Airin.docx                                                                22103233   
SafinAhmedAkash.docx                                                      22103019   
Md.RanaMiah(22103337).docx                                                22103337   
22103066LabExam(2).docx                                                   22103062   
22103066LabExam.docx                                                      22103066   
Mahia-22103118.pdf                                                        22103118   
Alvee_22103120_SecB.pdf                                                   22103120   
22103285(shanto).pdf                                  